<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/EEC_elmo_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
t = np.transpose
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
##Install flair

!pip install -q flair allennlp

In [0]:
##Using flair to load emlo small embeddinngs, which is in 768 dimensions
from flair.embeddings import BertEmbeddings, ELMoEmbeddings
from flair.data import Sentence


# init embedding
embedding = ELMoEmbeddings('small') #BertEmbeddings('bert-large-uncased')


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
##Load EEC Data
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
!unzip Equity-Evaluation-Corpus.zip
##Load Packages
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

##Read EEC
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!gdown https://drive.google.com/uc?id=1JyRVLZb7IdiMGGwPfXuA1f-CmCvxHDXx

Downloading...
From: https://drive.google.com/uc?id=1JyRVLZb7IdiMGGwPfXuA1f-CmCvxHDXx
To: /content/bert_conceptors.tar.bz2
656MB [00:06, 95.7MB/s]


In [0]:
!tar xvjf /content/bert_conceptors.tar.bz2

bert/
bert/bert_WEFAT_1_Male_attributes.pkl
bert/big_bert_gender_list_propernouns.pkl
bert/bert_W_5_African_American_names.pkl
bert/bert_W_2_Weapons.pkl
bert/bert_W_1_Pleasant.pkl
bert/bert_W_5_European_American_names.pkl
bert/bert_W_5_Unpleasant.pkl
bert/bert_W_6_Male_names.pkl
bert/big_bert_gender_list_all.pkl
bert/bert_W_10_Pleasant.pkl
bert/bert_W_1_Insects.pkl
bert/bert_W_5_Unused_full_list_European_American_names.pkl
bert/bert_W_10_Young_peoples_names.pkl
bert/bert_W_6_Female_names.pkl
bert/bert_W_9_Physical_disease.pkl
bert/bert_W_7_Female_terms.pkl
bert/big_bert_gender_list_pronouns.pkl
bert/bert_W_3_Unused_full_list_European_American_names.pkl
bert/bert_W_4_Unused_full_list_African_American_names.pkl
bert/bert_W_8_Male_terms.pkl
bert/bert_WEFAT_2_Androgynous_Names.pkl
bert/bert_WEFAT_1_Careers.pkl
bert/bert_W_9_Temporary.pkl
bert/bert_W_2_Unpleasant.pkl
bert/bert_WEFAT_1_Female_attributes.pkl
bert/bert_W_2_Instruments.pkl
bert/bert_W_7_Male_terms.pkl
bert/bert_W_3_Unpleasant.p

In [0]:
import pickle

In [0]:
embedding = BertEmbeddings('bert-large-uncased')

In [0]:
!gdown https://drive.google.com/uc?id=1WJBoEHOBkG4EW1MkB0y1QpggtV_J0i9w

Downloading...
From: https://drive.google.com/uc?id=1WJBoEHOBkG4EW1MkB0y1QpggtV_J0i9w
To: /content/elmo_race_list.pkl
4.88MB [00:00, 76.1MB/s]


In [0]:
x = pickle.load(open('/content/elmo_race_list.pkl','rb'))
#x.keys()
#dict_keys(['GnegC', 'type_embedings', 'words'])
negC = x['GnegC']
negC = np.squeeze(np.asarray(negC))


In [0]:
x = pickle.load(open('/content/bert/big_bert_race_list.pkl','rb'))
#x.keys()
#dict_keys(['GnegC', 'type_embedings', 'words'])
negC = x['GnegC']
negC = np.squeeze(np.asarray(negC))

In [0]:
negC.shape

(4096, 4096)

In [0]:
np.squeeze(np.asarray(negC))

array([[ 9.70111580e-01,  1.00139318e-02,  3.97474661e-03, ...,
         5.52274314e-03,  1.44466161e-03, -4.94146570e-03],
       [ 1.00139318e-02,  9.69216160e-01,  7.45507291e-03, ...,
         2.84306328e-04,  1.12427667e-03,  3.94247681e-03],
       [ 3.97474661e-03,  7.45507291e-03,  9.61890354e-01, ...,
        -2.32863467e-03, -4.04935310e-03,  1.12618329e-03],
       ...,
       [ 5.52274314e-03,  2.84306328e-04, -2.32863467e-03, ...,
         7.32905572e-01, -1.47071199e-02, -2.83304199e-02],
       [ 1.44466161e-03,  1.12427667e-03, -4.04935310e-03, ...,
        -1.47071199e-02,  9.38455688e-01,  5.55991601e-03],
       [-4.94146570e-03,  3.94247681e-03,  1.12618329e-03, ...,
        -2.83304199e-02,  5.55991601e-03,  7.92946442e-01]])

In [0]:
!gdown https://drive.google.com/uc?id=1dBiXlxSxfKq0-OhFQyL_CCij8pmThQZI

Downloading...
From: https://drive.google.com/uc?id=1dBiXlxSxfKq0-OhFQyL_CCij8pmThQZI
To: /content/bert_EEC_sentiment.pkl
100% 65.8k/65.8k [00:00<00:00, 34.1MB/s]


In [0]:
senn = pickle.load(open('/content/bert_EEC_sentiment.pkl','rb'))


In [0]:
senn['words']

['fear', 'sadness', 'joy', 'anger']

# Racial Debiasing

In [0]:
##Load male names in the EEC paper
word_list_male = ['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack']

/usr/local/lib/python2.7/dist-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=1JyRVLZb7IdiMGGwPfXuA1f-CmCvxHDXx
  .format(url='https://drive.google.com/uc?id={}'.format(file_id))
Downloading...
From: https://drive.google.com/file/d/1JyRVLZb7IdiMGGwPfXuA1f-CmCvxHDXx/view
To: /content/view
136kB [00:00, 8.74MB/s]


In [0]:
## Get racial names from web
!wget https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
!wget https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
!wget https://gist.githubusercontent.com/mbejda/26ad0574eda7fca78573/raw/6936d1a8f5fa5220f2f60a51a06a35b172c50f93/White-Female-Names.csv
!wget https://gist.githubusercontent.com/mbejda/9dc89056005a689a6456/raw/bb6ef2375f1289d0ef10dbd8e9469670ac23ceab/Black-Female-Names.csv


--2019-03-03 16:34:10--  https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921600 (900K) [text/plain]
Saving to: ‘Black-Male-Names.csv’

Black-Male-Names.cs 100%[===================>] 900.00K  --.-KB/s    in 0.07s   

2019-03-03 16:34:10 (11.9 MB/s) - ‘Black-Male-Names.csv’ saved [921600/921600]

--2019-03-03 16:34:10--  https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)

# Load racial names from web

In [0]:
## Load names from web, don't run it if using EEC names
word_list_male =[]

In [0]:
f = open('/content/White-Male-Names.csv', 'r+')
f.readline()
i=0
for line in f:
  i +=1
  word_list_male.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>=2400:
    break
f.close()

In [0]:
f = open('/content/Black-Male-Names.csv', 'r+')
i=0
f.readline()
for line in f:
  i+=1
  word_list_male.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>=2400:
    break
f.close()

In [0]:
word_list_female = []
f = open('/content/White-Female-Names.csv', 'r+')
f.readline()
i = 0
for line in f:
  i+=1
  word_list_female.append(line.rstrip().split(',')[1].split(' ')[1])
  if i >=2400:
    break
f.close()

In [0]:
f = open('/content/Black-Female-Names.csv', 'r+')
f.readline()
i = 0
for line in f:
  i+=1
  word_list_female.append(line.rstrip().split(',')[1].split(' ')[1])
  if i>=2400:
    break
f.close()

In [0]:
len(word_list_male), len(word_list_female)

(4800, 4800)

# Get CN embeddings

In [0]:
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

In [0]:
## Get brown_emb 
!gdown https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU
!pip3 install -q pymagnitude
from pymagnitude import *
from pymagnitude import MagnitudeUtils
import numpy as np
import pickle
with open('/content/elmo_embeddings_emma_brown.pkl', 'rb') as f:
    elmo_data = pickle.load(f)
elmo_data = elmo_data['brown_embs']

## Get brown corpus
import nltk
nltk.download('brown')
from nltk.corpus import brown
brown_corpus = brown.sents()

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [0]:
def pick_embeddings(corpus, sent_embs, word_list):
    X = []
    #labels = []
    #sents = []
    for i, s in enumerate(corpus):
       # print(s)
        for j, w in enumerate(s):
            if w in word_list:
                X.append(sent_embs[i][j])
                #labels.append(w)
                #sents.append(s)
    return X


In [0]:
## Get elmo word embeddings of words(male names) to find conceptors
X = pick_embeddings(brown_corpus, elmo_data, word_list_male)
cn_embeddings = np.array(X)

In [0]:
## Since conceptors doesn't change in the loop, I calculated conceptor negation matrix beforehand to reduce runtime.
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

In [0]:
sent1_emb

array([], shape=(1, 0), dtype=float32)

# Run experiment-ELMO


In [0]:
def run_raw(sen_emb, sent1, sent2):
  
  sentence1 = Sentence(sent1)
  sentence2 = Sentence(sent2)
  x = embedding.embed(sentence1) # get sentence embedding of sentence 1
  se = torch.stack([token.embedding for token in sentence1])
  sent1_emb = se.numpy()
  x = embedding.embed(sentence2) # get sentence embedding of sentence 2
  se = torch.stack([token.embedding for token in sentence2])
  sent2_emb = se.numpy()
  sent1_emb = np.average(sent1_emb, axis = 0) #average sentence1 embeddings
  sent2_emb = np.average(sent2_emb, axis = 0) #average sentence2 embeddings
  proj1 = (sent1_emb/norm(sent1_emb)).dot(sen_emb)/norm(sen_emb) #cosine similarity
  proj2 = (sent2_emb/norm(sent2_emb)).dot(sen_emb)/norm(sen_emb)
  #print(sent2_emb)
  #print(proj1, proj2)
  return proj1, proj2

In [0]:
def run_conceptor(sen_emb, sent1, sent2):
  sentence1 = Sentence(sent1) 
  sentence2 = Sentence(sent2)
  x = embedding.embed(sentence1) # get sentence embedding of sentence 1
  se = torch.stack([token.embedding for token in sentence1])
  sent1_emb = se.numpy()
  x = embedding.embed(sentence2) # get sentence embedding of sentence 2
  se = torch.stack([token.embedding for token in sentence2])
  sent2_emb = se.numpy()
  sent1_emb = np.average(sent1_emb, axis = 0) #average sentence1 embeddings
  sent2_emb = np.average(sent2_emb, axis = 0) #average sentence2 embeddings
  #print(sent1_emb.T.shape)
  sent1_emb = negC.dot(sent1_emb.T).T  # CN sentence embeddings
  #print(sent1_emb.shape)
  sent2_emb = negC.dot(sent2_emb.T).T
  sen_emb =  negC.dot(sen_emb.T).T
  #print(sent2_emb)
  proj1 = (sent1_emb/norm(sent1_emb)).dot(sen_emb)/norm(sen_emb) #cosine similarity
  proj2 = (sent2_emb/norm(sent2_emb)).dot(sen_emb)/norm(sen_emb)
  #print(proj1,proj2)
  return proj1, proj2

In [0]:
EEC[EEC['Race']=='European'].count()

ID              2880
Sentence        2880
Template        2880
Person          2880
Gender          2880
Race            2880
Emotion         2800
Emotion word    2800
dtype: int64

In [0]:
# Get male sentences results
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_aa = sent_sent[(sent_sent['Race']=='African-American')] #get AA male sentences
  sent_ea = sent_sent[(sent_sent['Race']=='European')] #get EA male sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    aa.append(score1) # raw score for african american
    ea.append(score2) # raw score for european american
    aa_cn.append(score3) # CN score for african american
    ea_cn.append(score4) # CN score for european american
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(aa, ea)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


700
700
700
700


In [0]:
print('RAW vs CN data using ELMO, racial debiasing')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


RAW vs CN data using ELMO, racial debiasing
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          -6.437485  2.256846e-10      -11.237026  4.765867e-27
fear           -4.731632  2.696051e-06       -9.460335  4.574093e-20
joy            -5.928435  4.804908e-09       -3.817702  1.466543e-04
sadness        -4.916008  1.101632e-06       -9.110037  8.550151e-19


#Bert

In [0]:
# Get male sentences results
p= []
i = 0
for sen in ['fear','sadness','joy', 'anger']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_aa = sent_sent[(sent_sent['Race']=='African-American')] #get AA male sentences
  sent_ea = sent_sent[(sent_sent['Race']=='European')] #get EA male sentences
  sen_emb = np.squeeze(np.asarray(senn['type_embedings'][i]))
  i+=1
  print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    aa.append(score1) # raw score for african american
    ea.append(score2) # raw score for european american
    aa_cn.append(score3) # CN score for african american
    ea_cn.append(score4) # CN score for european american
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(aa, ea)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


700
700
700
700


In [0]:
print('RAW vs CN data using BERT, racial debiasing')
p = pd.DataFrame(p, index=['fear','sadness','joy','anger'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using BERT, racial debiasing
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
fear            5.647380  2.370618e-08       22.624530  1.853297e-85
sadness        -3.449800  5.946922e-04        3.267547  1.137887e-03
joy             4.756097  2.398224e-06        9.040590  1.512630e-18
anger           1.633738  1.027643e-01       13.350301  2.237348e-36


In [0]:
# Load female names in EEC paper, don't run it if using web names
word_list_female= ['Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen']

In [0]:
## Get elmo word embeddings of words(female names) to find conceptors
X = pick_embeddings(brown_corpus, elmo_data,word_list_female)
cn_embeddings = np.array(X)

In [0]:
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

In [0]:
## Get female sentences result
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')] #get AA female sentences
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ] #get EA female sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    aa.append(score1) # raw score for african american
    ea.append(score2) # raw score for european american
    aa_cn.append(score3) # CN score for african american
    ea_cn.append(score4) # CN score for european american
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(aa, ea)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


350
350
350
350


In [0]:
print('RAW vs CN data using ELMO, female names from EEC, alpha = 1')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


RAW vs CN data using ELMO, female names from EEC, alpha = 1
         raw_t_statistic  raw_p_value  cn_t_statistic  cn_p_value
anger          -1.530739     0.126740        0.229301    0.818769
fear           -1.382389     0.167736        0.573728    0.566521
joy             0.668064     0.504534        0.103622    0.917529
sadness        -1.684079     0.093060        0.576262    0.564810


# Gender Debiasing

# Load gender names

# Using names from web

In [0]:
## Get large gender name lists from web, don't run it if using names only in EEC paper
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt

--2019-03-03 01:07:34--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20466 (20K) [text/plain]
Saving to: ‘male.txt’

male.txt            100%[===================>]  19.99K  --.-KB/s    in 0s      

2019-03-03 01:07:34 (197 MB/s) - ‘male.txt’ saved [20466/20466]

--2019-03-03 01:07:36--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt’

female.txt          100%[===================>]  34.91K  --.-KB/s    in 0.04s   

2019-03-03 01:07:36 (938 KB/s) - ‘female.txt’ saved [35751/35751]



In [0]:
f = open('/content/male.txt', 'r+')
i = 0
word_list_gender= []
for line in f:
  i+=1
  if i < 7:
    continue
  if i >=2407:
    break
  token = line.split()
  word_list_gender.append(token[0])
f.close()
#load names from larger name dataset
f = open('/content/female.txt', 'r+')
i = 0
for line in f:
  i+=1
  if i < 7:
    continue
  if i>=2407:
    break
  token = line.split()
  word_list_gender.append(token[0])
f.close()

In [0]:
len(word_list_gender)

4800

In [0]:
X = pick_embeddings(brown_corpus, elmo_data,word_list_gender)
cn_embeddings = np.array(X)
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

In [0]:
## Get female sentences result
p= []
for sen in ['anger','fear','joy', 'sadness']:
  ma = []
  fe = []
  ma_cn = []
  fe_cn = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_ma = sent_sent[(sent_sent['Gender']=='male')] #get male sentences
  sent_fe = sent_sent[(sent_sent['Gender']=='female') ] #get female sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  for sen1, sen2 in zip(sent_ma['Sentence'], sent_fe['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    ma.append(score1) # raw score for male 
    fe.append(score2) # raw score for female
    ma_cn.append(score3) # CN score for male
    fe_cn.append(score4) # CN score for female
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(ma, fe)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(ma_cn, fe_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [0]:
print('Gender Debiasing, RAW vs CN data using ELMO, using 8k gender names, alpha = 1')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


Gender Debiasing, RAW vs CN data using ELMO, using 8k gender names, alpha = 1
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger           2.546940  1.100909e-02       -6.030878  2.257000e-09
fear           -0.861153  3.893503e-01       -6.045076  2.073065e-09
joy            -7.560720  8.726120e-14       11.623958  1.824004e-29
sadness        -0.986189  3.242680e-01       -6.234039  6.576155e-10


In [0]:
EEC['Person'].unique()

array(['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'he',
       'this man', 'this boy', 'my brother', 'my son', 'my husband',
       'my boyfriend', 'my father', 'my uncle', 'my dad', 'Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen', 'she', 'this woman', 'this girl', 'my sister',
       'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
       'my mom', 'him', 'her'], dtype=object)

# Using names in paper

In [0]:
# Names in EEC paper
word_list_gender = EEC['Person'].unique()

In [0]:
len(word_list_gender)

62

In [0]:
X = pick_embeddings(brown_corpus, elmo_data,word_list_gender)
cn_embeddings = np.array(X)
alpha = 1
R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]   
C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(768))# calculate the conceptor matrix
negC = np.eye(768) - C

In [0]:
## Get female sentences result
p= []
for sen in ['anger','fear','joy', 'sadness']:
  ma = []
  fe = []
  ma_cn = []
  fe_cn = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_ma = sent_sent[(sent_sent['Gender']=='male')] #get male sentences
  sent_fe = sent_sent[(sent_sent['Gender']=='female') ] #get female sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  for sen1, sen2 in zip(sent_ma['Sentence'], sent_fe['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    ma.append(score1) # raw score for male
    fe.append(score2) # raw score for female
    ma_cn.append(score3) # CN score for male
    fe_cn.append(score4) # CN score for female
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(ma, fe)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(ma_cn, fe_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [0]:
print('Gender Debiasing, RAW vs CN data using ELMO, using 62 gender names and pronouns in EEC, alpha = 1')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

Gender Debiasing, RAW vs CN data using ELMO, using 62 gender names and pronouns in EEC, alpha = 1
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger           2.546525  1.102212e-02       -6.406935  2.238108e-10
fear           -0.861151  3.893519e-01       -8.237408  5.199203e-16
joy            -7.560721  8.726056e-14       -6.488792  1.331468e-10
sadness        -0.986190  3.242674e-01       -8.491553  6.893392e-17
